# Create 2D Binary Models with Elliptical Inclusions

This notebook demonstrates the `create_binary_model_2d` function for generating synthetic 2D binary models with elliptical inclusions. These models are useful for:

- **2D microstructure analysis** (thin sections, cross-sections)
- **Image processing algorithm development** and testing
- **Fast prototyping** before moving to 3D
- **Educational demonstrations** of digital rock physics concepts
- **Slice-based analysis** workflows

## Key Features

- **Dedicated 2D function**: `create_binary_model_2d(nx, ny, ...)`
- **Efficient 2D operations**: No unnecessary 3D overhead
- **Shape control**: Circles, ellipses (elongated or flattened)
- **Optional rotation**: 2D rotation in the plane
- **Periodic boundaries**: Seamless tiling for RVE applications
- **Output format**: (nx, ny, 1) array for compatibility with 3D pipelines

## Import Required Libraries

In [ ]:
from drp_template.model import binary_2d
from drp_template.input_output import export_model
from drp_template.image import ortho_views, save_figure
from drp_template.math import get_phase_fractions
import numpy as np
import matplotlib.pyplot as plt

## Example 1: Single Circular Pore

Create a simple 2D model with one circular pore inclusion centered in the image.

In [ ]:
# Create 2D model with centered circle
nx, ny = 200, 200
center_position = np.array([[nx // 2, ny // 2]])  # Centered at (100, 100)

data = binary_2d(
    nx=nx, ny=ny,
    num_inclusions=1,
    inclusion_radius=40,
    inclusion_aspect_ratio=1.0,  # Circle
    random_orientation=False,
    # Default: background_value=1 (Solid), inclusion_value=0 (Pore)
    dtype='uint8',
    positions=center_position
)

print(f"Created 2D model with shape: {data.shape}")
print(f"Note: Shape is (nx, ny, 1) for compatibility with 3D pipelines")
print(f"Unique values: {np.unique(data)}")
print(f"Center position: {center_position[0]}")

# Visualize
plt.figure(figsize=(8, 8))
plt.imshow(data[:, :, 0].T, cmap='gray', origin='lower', vmin=0, vmax=1)
plt.plot(center_position[0, 0], center_position[0, 1], 'r+', markersize=15, markeredgewidth=2)
plt.title('Single Circular Pore (Centered)')
plt.xlabel('x (pixels)')
plt.ylabel('y (pixels)')
plt.colorbar(label='Phase (0=Pore, 1=Solid)')
plt.tight_layout()
plt.show()

# Export and analyze
labels = {0: 'Pore', 1: 'Solid'}
export_model(
    filename='binary_2d_circle',
    data=data,
    voxel_size=10.0,
    dtype='uint8',
    labels=labels
)
get_phase_fractions(data, labels=labels, log=True)

## Example 2: Multiple Elliptical Pores

Create a model with multiple elliptical pores with random orientations (2D rotation).

In [ ]:
# Create 2D model with multiple elongated ellipses
data = binary_2d(
    nx=300, ny=300,
    num_inclusions=20,
    inclusion_radius=25,
    inclusion_aspect_ratio=2.5,  # Elongated ellipse
    random_orientation=True,  # Random 2D rotation
    dtype='uint8',
    seed=42
)

print(f"Created 2D model with shape: {data.shape}")
print(f"Number of inclusions: 20")
print(f"Aspect ratio: 2.5 (elongated)")
print(f"Random orientation: True (2D rotation)")

# Visualize
plt.figure(figsize=(10, 10))
plt.imshow(data[:, :, 0].T, cmap='gray', origin='lower', vmin=0, vmax=1)
plt.title('Multiple Elliptical Pores with Random Orientations')
plt.xlabel('x (pixels)')
plt.ylabel('y (pixels)')
plt.colorbar(label='Phase (0=Pore, 1=Solid)')
plt.tight_layout()
plt.show()

# Export and analyze
labels = {0: 'Pore', 1: 'Rock Matrix'}
export_model(
    filename='binary_2d_ellipses',
    data=data,
    voxel_size=5.0,
    dtype='uint8',
    labels=labels
)
get_phase_fractions(data, labels=labels, log=True)

## Example 3: Periodic Boundary Conditions (2D RVE)

Periodic boundary conditions in 2D create seamless, tileable images where inclusions near edges wrap to the opposite side. This is essential for:

- **2D Representative Volume Elements (RVE)**
- **Seamless texture generation**
- **Finite element simulations** with periodic BCs
- **Avoiding edge artifacts** in statistical analysis

In [ ]:
# Create model with large inclusion near corner WITHOUT periodic boundaries
nx, ny = 150, 150
corner_position = np.array([[20, 20]])  # Near bottom-left corner

data_no_pbc = binary_2d(
    nx=nx, ny=ny,
    num_inclusions=1,
    inclusion_radius=40,  # Large enough to extend beyond boundaries
    inclusion_aspect_ratio=1.0,
    random_orientation=False,
    periodic=False,  # No periodic boundaries
    dtype='uint8',
    positions=corner_position
)

print(f"Model shape: {data_no_pbc.shape}")
print(f"Inclusion position: {corner_position[0]}")
print(f"Inclusion radius: 40 pixels")
print(f"Without PBC: Inclusion is cut off at boundaries")

# Check boundary pixels
left_edge = data_no_pbc[0, :, 0]
right_edge = data_no_pbc[-1, :, 0]
top_edge = data_no_pbc[:, -1, 0]
bottom_edge = data_no_pbc[:, 0, 0]

print(f"\nBoundary analysis:")
print(f"  Pores on left edge:   {np.sum(left_edge == 0)}/{len(left_edge)}")
print(f"  Pores on right edge:  {np.sum(right_edge == 0)}/{len(right_edge)}")
print(f"  Pores on bottom edge: {np.sum(bottom_edge == 0)}/{len(bottom_edge)}")
print(f"  Pores on top edge:    {np.sum(top_edge == 0)}/{len(top_edge)}")
print(f"  → Inclusion only on left/bottom edges (cut off, no wrapping)")

# Visualize
plt.figure(figsize=(8, 8))
plt.imshow(data_no_pbc[:, :, 0].T, cmap='gray', origin='lower', vmin=0, vmax=1)
plt.plot(corner_position[0, 0], corner_position[0, 1], 'r+', markersize=15, markeredgewidth=2)
plt.title('No Periodic BC: Inclusion Cut Off at Boundaries')
plt.xlabel('x (pixels)')
plt.ylabel('y (pixels)')
plt.colorbar(label='Phase (0=Pore, 1=Solid)')
plt.tight_layout()
plt.show()

In [ ]:
data_pbc = binary_2d(
    nx=nx, ny=ny,
    num_inclusions=1,
    inclusion_radius=40,
    inclusion_aspect_ratio=1.0,
    random_orientation=False,
    periodic=True,  # Enable periodic boundaries
    dtype='uint8',
    positions=corner_position
)

print(f"Model shape: {data_pbc.shape}")
print(f"With PBC: Inclusion wraps to opposite boundaries")

# Check all 4 boundary edges
left_edge_pbc = data_pbc[0, :, 0]
right_edge_pbc = data_pbc[-1, :, 0]
top_edge_pbc = data_pbc[:, -1, 0]
bottom_edge_pbc = data_pbc[:, 0, 0]

print(f"\nBoundary analysis:")
print(f"  Pores on left edge:   {np.sum(left_edge_pbc == 0)}/{len(left_edge_pbc)}")
print(f"  Pores on right edge:  {np.sum(right_edge_pbc == 0)}/{len(right_edge_pbc)} ✓ (wrapped)")
print(f"  Pores on bottom edge: {np.sum(bottom_edge_pbc == 0)}/{len(bottom_edge_pbc)}")
print(f"  Pores on top edge:    {np.sum(top_edge_pbc == 0)}/{len(top_edge_pbc)} ✓ (wrapped)")
print(f"\n✅ Inclusion appears on all 4 boundary edges (seamless wrapping)")

# Visualize
plt.figure(figsize=(8, 8))
plt.imshow(data_pbc[:, :, 0].T, cmap='gray', origin='lower', vmin=0, vmax=1)
plt.plot(corner_position[0, 0], corner_position[0, 1], 'r+', markersize=15, markeredgewidth=2)
plt.title('With Periodic BC: Inclusion Wraps to Opposite Edges')
plt.xlabel('x (pixels)')
plt.ylabel('y (pixels)')
plt.colorbar(label='Phase (0=Pore, 1=Solid)')
plt.tight_layout()
plt.show()

### Demonstrating 2×2 Seamless Tiling

In [ ]:
# Create a 2×2 tiled version to demonstrate seamless boundaries
tiled = np.block([
    [data_pbc[:, :, 0], data_pbc[:, :, 0]],
    [data_pbc[:, :, 0], data_pbc[:, :, 0]]
])

print(f"Original size: {data_pbc.shape}")
print(f"Tiled 2×2 size: {tiled.shape}")

# Visualize side-by-side comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Original without PBC
axes[0].imshow(data_no_pbc[:, :, 0].T, cmap='gray', origin='lower', vmin=0, vmax=1)
axes[0].set_title('Without PBC\n(Inclusion cut off at boundaries)')
axes[0].set_xlabel('x (pixels)')
axes[0].set_ylabel('y (pixels)')
axes[0].plot(corner_position[0, 0], corner_position[0, 1], 'r+', markersize=15, markeredgewidth=2)

# Original with PBC
axes[1].imshow(data_pbc[:, :, 0].T, cmap='gray', origin='lower', vmin=0, vmax=1)
axes[1].set_title('With PBC\n(Inclusion wraps to opposite edges)')
axes[1].set_xlabel('x (pixels)')
axes[1].set_ylabel('y (pixels)')
axes[1].plot(corner_position[0, 0], corner_position[0, 1], 'r+', markersize=15, markeredgewidth=2)

# 2×2 tiling
axes[2].imshow(tiled.T, cmap='gray', origin='lower', vmin=0, vmax=1)
axes[2].set_title('2×2 Tiling\n(Seamless boundaries)')
axes[2].set_xlabel('x (pixels)')
axes[2].set_ylabel('y (pixels)')
# Mark the tile boundaries
axes[2].axhline(ny, color='red', linewidth=2, linestyle='--', alpha=0.7, label='Tile boundaries')
axes[2].axvline(nx, color='red', linewidth=2, linestyle='--', alpha=0.7)
axes[2].legend()

plt.tight_layout()
plt.show()

print("\n✅ Notice: No visible seams at the red boundary lines!")
print("   The periodic wrapping creates perfectly seamless tiling.")

In [ ]:
# Create 2D RVE with multiple random inclusions
data_rve = binary_2d(
    nx=200, ny=200,
    num_inclusions=15,
    inclusion_radius=20,
    inclusion_aspect_ratio=1.5,  # Slightly elongated
    random_orientation=True,  # Random 2D rotation
    periodic=True,  # Periodic boundaries for RVE
    dtype='uint8',
    seed=123
)

print(f"2D RVE shape: {data_rve.shape}")
print(f"Number of inclusions: 15")
print(f"Periodic boundaries: True")

# Export
labels = {0: 'Pore', 1: 'Rock Matrix'}
export_model(
    filename='binary_2d_rve_periodic',
    data=data_rve,
    voxel_size=5.0,
    dtype='uint8',
    labels=labels
)
get_phase_fractions(data_rve, labels=labels, log=True)

# Visualize
plt.figure(figsize=(10, 10))
plt.imshow(data_rve[:, :, 0].T, cmap='gray', origin='lower', vmin=0, vmax=1)
plt.title('2D RVE with Periodic Boundaries\n(15 randomly oriented ellipses)')
plt.xlabel('x (pixels)')
plt.ylabel('y (pixels)')
plt.colorbar(label='Phase (0=Pore, 1=Solid)')
plt.tight_layout()
plt.show()

print("\n✅ This 2D RVE can be seamlessly tiled in all directions!")

## Parameter Summary

Key parameters for `binary_2d`:

| Parameter | Values | Effect |
|-----------|--------|--------|
| `nx, ny` | `int` | Image dimensions (pixels) |
| `num_inclusions` | `int` | Number of elliptical inclusions |
| `inclusion_radius` | `float` | Radius in pixels (major axis for ellipses) |
| `inclusion_aspect_ratio` | `1.0` | Circle |
| | `< 1.0` | Flattened ellipse |
| | `> 1.0` | Elongated ellipse |
| `random_orientation` | `True` / `False` | Random 2D rotation in plane |
| `periodic` | `True` / `False` | Enable periodic boundary conditions |
| `background_value` | `1` (default) | Solid phase |
| `inclusion_value` | `0` (default) | Pore phase |
| `positions` | `None` | Random placement |
| | `array (N, 2)` | Explicit [x, y] coordinates |
| `seed` | `int` | Reproducible random placement |
| `dtype` | `'uint8'` | Data type for output |

**Output Shape**: Always `(nx, ny, 1)` for compatibility with 3D pipelines

**Periodic Boundaries**: When enabled, inclusions wrap around edges (up to 9 positions: 1 original + 4 edges + 4 corners)

## Summary

This notebook demonstrated:

✅ **Creating 2D binary models** with the dedicated `binary_2d()` function  
✅ **Shape control** via aspect ratio (circles, elongated/flattened ellipses)  
✅ **Random 2D rotation** for varied orientations  
✅ **Periodic boundary conditions** for seamless tiling (RVE applications)  
✅ **Explicit positioning** for precise spatial control  
✅ **Export with metadata** and phase fraction analysis  
✅ **Comparison with 3D** models and when to use each  

### Key Takeaways

1. **Efficient 2D operations**: `binary_2d` is optimized for 2D without 3D overhead
2. **Periodic boundaries**: Essential for RVE applications and seamless tiling
3. **Output format**: `(nx, ny, 1)` shape maintains compatibility with 3D pipelines
4. **2D rotation**: Simple in-plane rotation (not full 3D Euler angles)

### Next Steps

- Try with different aspect ratios and orientations
- Experiment with periodic boundaries for texture generation
- Scale up to 3D using `binary_3d` when needed
- Use for validating image segmentation algorithms
- Generate training data for machine learning

### See Also

- [3D Binary Models Notebook](binary_3d.ipynb)
- [2D vs 3D Models Guide](../../docs/guides/2d_vs_3d_models.md)
- [Tools API Reference](../../docs/api/tools.md)